In [12]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='kyjin0413!', db='lpointsample', charset='utf8')
cur = conn.cursor()
cur.execute("select * from 구매점수_소분류;")

result = cur.fetchall()

conn.commit()
conn.close()

df = pd.DataFrame(result)
df

,0,1,2,3
0,M000034966,가공조미료,3,9360
1,M000034966,간장,1,8300
2,M000034966,감자스낵,2,4360
3,M000034966,과일음료,1,1200
4,M000034966,국산맥주,6,27240
...,...,...,...,...
1631356,M999962961,호두,1,9990
1631357,M999962961,호주산소우둔,1,17800
1631358,M999962961,혼합견과,1,2980
1631359,M999962961,혼합잡곡,1,7990


In [15]:
# df.set_index(0, inplace=True)
# df.head()

,1,2,3
0,,,
M000034966,가공조미료,3,9360
M000034966,간장,1,8300
M000034966,감자스낵,2,4360
M000034966,과일음료,1,1200
M000034966,국산맥주,6,27240


In [25]:
df.columns = ['cust', '소분류', '수량', '금액']
df

,cust,소분류,수량,금액
0,M000034966,가공조미료,3,9360
1,M000034966,간장,1,8300
2,M000034966,감자스낵,2,4360
3,M000034966,과일음료,1,1200
4,M000034966,국산맥주,6,27240
...,...,...,...,...
1631356,M999962961,호두,1,9990
1631357,M999962961,호주산소우둔,1,17800
1631358,M999962961,혼합견과,1,2980
1631359,M999962961,혼합잡곡,1,7990


In [26]:
# 사용자 유사도
df_pivot4user = df.pivot('cust','소분류','수량')
# 품목별 유사도
df_pivot4category = df.pivot('소분류', 'cust', '수량')

In [27]:
df_pivot4user=df_pivot4user.fillna(0)

In [28]:
df_pivot4category=df_pivot4category.fillna(0)

In [29]:
# 사용자 기준
cos_user = cosine_similarity(df_pivot4user, df_pivot4user)

In [30]:
# 위에서 만든 코사인 유사도 행렬 데이터 프레임으로 변환
df_cos = pd.DataFrame(data = cos_user, index=df_pivot4user.index, columns=df_pivot4user.index)
df_cos.head()

cust,M000034966,M000136117,M000201112,M000225114,M000261625,M000350564,M000419293,M000494848,M000504230,M000508243,...,M999340261,M999441814,M999468256,M999492154,M999515910,M999599111,M999673157,M999770689,M999849895,M999962961
cust,,,,,,,,,,,,,,,,,,,,,
M000034966,1.000000,0.054006,0.281184,0.190258,0.043164,0.000000,0.148297,0.089548,0.501796,0.044281,...,0.000000,0.000000,0.052793,0.00000,0.137970,0.138257,0.000000,0.051292,0.040454,0.081385
M000136117,0.054006,1.000000,0.025031,0.050616,0.146737,0.015284,0.375709,0.150981,0.046606,0.030747,...,0.026207,0.000000,0.006788,0.42521,0.091186,0.017778,0.116501,0.037990,0.007124,0.075902
M000201112,0.281184,0.025031,1.000000,0.056761,0.000000,0.000000,0.000000,0.053849,0.553858,0.057466,...,0.000000,0.093659,0.004758,0.00000,0.242911,0.099681,0.056802,0.104841,0.022826,0.097853
M000225114,0.190258,0.050616,0.056761,1.000000,0.205309,0.017329,0.283987,0.221068,0.040683,0.005810,...,0.000000,0.000000,0.056283,0.00000,0.301580,0.254478,0.025844,0.134267,0.058273,0.169290
M000261625,0.043164,0.146737,0.000000,0.205309,1.000000,0.049881,0.149216,0.078722,0.005192,0.007168,...,0.000000,0.000000,0.021364,0.00000,0.058743,0.049732,0.170033,0.087176,0.032741,0.264247


In [45]:
# 유사도 높은 사용자 10명 추출
def sim_user(user_id):
  return df_cos[user_id].sort_values(ascending=False)[1:11]

In [65]:
sim_user1 = pd.DataFrame(sim_user('M000136117'))
sim_user1.reset_index(drop=False, inplace=True)
sim_user1 = df[sim_user1.loc[0]['cust'] == df['cust']]
sim_user1[['소분류','수량']].sort_values(by= '수량', ascending=False)[0:3]

,소분류,수량
570877,기타남성의류세트,8
570879,기타여성속옷,7
570881,남성스포츠패딩,2


In [ ]:
df1000=df[df['cust']=='M346999304']
df1000

,cust,소분류,수량,금액
570877,M346999304,기타남성의류세트,8,78400
570878,M346999304,기타모자,1,30000
570879,M346999304,기타여성속옷,7,113000
570880,M346999304,기타패션잡화,1,159000
570881,M346999304,남성스포츠패딩,2,130600
570882,M346999304,수예소모품,1,44910
570883,M346999304,영양바,1,56000
570884,M346999304,일반스낵,1,16000
570885,M346999304,일식,1,42000
570886,M346999304,즉석젓갈류,2,30000


In [ ]:
df1000.sort_values(by= '수량', ascending=False)[0:3]

,cust,소분류,수량,금액
570877,M346999304,기타남성의류세트,8,78400
570879,M346999304,기타여성속옷,7,113000
570881,M346999304,남성스포츠패딩,2,130600


In [ ]:
df1000[['소분류','수량']].sort_values(by= '수량', ascending=False)[0:3]

,소분류,수량
570877,기타남성의류세트,8
570879,기타여성속옷,7
570881,남성스포츠패딩,2


In [ ]:
cos_category = cosine_similarity(df_pivot4category, df_pivot4category)

In [ ]:
df_cos = pd.DataFrame(data = cos_category, index=df_pivot4category.index, columns=df_pivot4category.index)
df_cos.head()

소분류,2단우산,3단우산,BB/파운데이션/컴팩트류,CDP/카세트,DIY가구,DIY완구,DSLR,DVD플레이어,LED,LED램프,...,후크류,훈련,훈제/구이오징어,훈제/로스트,휴대폰,휴대폰용품,휴지통,흑미,히터,힙색/사이드백
소분류,,,,,,,,,,,,,,,,,,,,,
2단우산,1.000000,0.042798,0.015037,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.005456,0.004263,0.003186,0.022222,0.002060,0.000000,0.000000,0.000000
3단우산,0.042798,1.000000,0.015931,0.00000,0.000000,0.018399,0.0,0.0,0.002231,0.002253,...,0.013860,0.000000,0.007854,0.006303,0.005206,0.012969,0.007213,0.024641,0.002786,0.000000
BB/파운데이션/컴팩트류,0.015037,0.015931,1.000000,0.00156,0.011156,0.024123,0.0,0.0,0.013930,0.036498,...,0.036769,0.002528,0.048970,0.015943,0.047096,0.030460,0.007529,0.028970,0.004970,0.005173
CDP/카세트,0.000000,0.000000,0.001560,1.00000,0.000000,0.002725,0.0,0.0,0.000000,0.000000,...,0.049267,0.000000,0.004904,0.000639,0.008592,0.044947,0.001852,0.000000,0.016093,0.000000
DIY가구,0.000000,0.000000,0.011156,0.00000,1.000000,0.002924,0.0,0.0,0.000000,0.023267,...,0.019823,0.000000,0.001754,0.000685,0.012291,0.010717,0.011588,0.000000,0.000000,0.000000


In [ ]:
# 유사도 높은 사용자 10명 추출
def sim_cate(user_id):
  return df_cos[user_id].sort_values(ascending=False)[1:11]

In [ ]:
sim_cate('기타남성의류세트')

소분류
한식           0.176938
기타패션잡화       0.176039
쿠키           0.164924
푸드코트한식       0.157298
여성정장세트       0.157245
여성숄더백        0.148747
기타여성의류아우터    0.145441
기타레토르트       0.145250
기타국산과일류      0.145060
양식           0.143305
Name: 기타남성의류세트, dtype: float64